In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
print(f"Project root added to path: {project_root}")


from pathlib import Path
from src.utils import fetch_data_from_bq
import pandas as pd
from src import sql_queries as q
import pandas as pd
import numpy as np
import json

Project root added to path: d:\My_Projects\OLIST\python


In [2]:
from src.context_builder import BusinessContextBuilder

# Initialize with your directory
builder = BusinessContextBuilder(reports_dir=r'D:\My_Projects\OLIST\python\outputs\Analysis')

# Load all reports
builder.load_all_reports()

# Generate and save the context
builder.save_context("business_context.txt")

# Display a preview in the notebook
context = builder.build_full_context()
print(context)

Business context saved to: D:\My_Projects\OLIST\python\outputs\Analysis\business_context.txt
Total length: 9,823 characters

████████████████████████████████████████████████████████████████████████████████
COMPREHENSIVE BUSINESS INTELLIGENCE REPORT
Generated: 2025-12-14 23:08:58
████████████████████████████████████████████████████████████████████████████████

EXECUTIVE SUMMARY

Business Overview:
- Total Revenue (including freight, etc.): $15,422,461.77
- Total Orders: 96,477
- Total Customers: 93,357
- Total Sellers: 2,970
- Total Items Ordered: 110,194

Key Performance Indicators:
- Average Order Value: $159.86
- Average Basket Size: 1.14 items
- Revenue per Customer: $165.20
- Orders per Customer: 1.03
- Revenue per Seller: $5,192.75
- Orders per Seller: 32.48

MONTHLY TRENDS & GROWTH

Time Period Overview:
- Analysis Period: 2016-10 to 2018-08
- Total Months: 22
- Average Monthly Revenue: $701,020.99
- Average Monthly Orders: 4,385.0

Last 12 Months Summary:
- Average Monthly Reven